### Quasiquote (backquote) `` ` ``, unquote `,` e unquote-splicing `,@`

Convenhamos que ficar escrevendo `(list` junto a `'sexp` para uma expressão que não queremos computar e somente `sexp` para uma expressão que queremos computar não é muito elegante.

Por conta disso, scheme vem equipado com um reader macro chamado de **quasiquote** ou **backquote** (``` ` ```), ele funciona de forma similar ao **quote**:

In [3]:
`(x y 1 2)

(x y 1 2)

Entretando, quando combinado com outro reader macros **unquote** (`,`) ele é capaz de computar valores sob demanda, por exemplo:

In [4]:
(define x 1)
(define y 2)

`((x ,x) (y ,y))

((x 1) (y 2))

### Questão 1

O que o código a seguir retorna? 

```scheme
(define x 'y)
`,x 
```

### Questão 2

E o código a seguir?
```scheme
(define x 'y)
`',x
```

Além disso, temos também o reader macro **unquote-splicing** (`,@`) que é meio complicado de entender, mas em poucas palavras ele "remove 1 par de parenteses de uma lista".

Ou seja:


In [9]:
(define x '(1 2 3 4))

`(0 ,@x)

(0 1 2 3 4)

Viram como a lista `'(1 2 3 4)` foi "desempacotada" em `1 2 3 4` para depois se juntar ao `0` em uma nova lista?

O **unquote-splicing** é poderosissimo e utilizado para trabalhar com macros que recebem um número váriavel de argumentos, por exemplo:

In [5]:
(define (f . args)
  `(begin ,@args))

(f '(display "oi")
   '(newline) 
   '(display "tchau") 
   '(newline))

(begin (display "oi") (newline) (display "tchau") (newline))

Ou seja, se `f` fosse um macro, em tempo de compilação, nós teriámos uma lista de listas: 

```scheme
'((display "oi")
  (newline)
  (display "tchau")
  (newline))
```

Que seria transformado em um código:

```scheme
(begin (display "oi") 
       (newline) 
       (display "tchau")
       (newline))
```

Como mostrado a seguir:

In [4]:
(define-macro (m . args)
  `(begin ,@args))

(m (display "oi")
   (newline) 
   (display "tchau") 
   (newline))

oi
tchau


Em resumo, utilizamos **unquote-splicing** para adicionar elementos no inicio ou no fim de uma série de instruções sem precisar utilizar funções que manipulam a lista.

#### Questão 3
Complete o macro `desafio`, utilizando **somente** **quasiquote**, **unquote** e **unquote-splicing** que recebe um argumento `arg1`, uma função `f`, e um número váriavel de argumentos `args` e retorna uma aplicação de `f` a `args` e `arg1`.

Exemplo:

```scheme
(desafio 2 / 1 3 5) 

;; expande para

(/ 1 3 5 2)

;; computa para

1/30
```

In [1]:
(define-macro (desafio arg1 f . args)
  `(? ?? ???))

(desafio 2 / 1 3 5)

1/30

### Qual a utilidade de macros?

Como vimos na última aula, macros são utilizados para manipular estruturas de uma linguagem, no caso do **scheme**, cons cells.

Por exemplo, um problema que encontramos recorrentemente em **scheme** é que `lambdas` não possuem nome, ou seja, é muito difícil em, um único bloco `lambda`, fazermos recursão.

Para resolvermos isso podemos criar o macro `nlambda` (named-lambda) que possui a seguinte estrutura:

```scheme
(nlambda (nome . args)
   corpo)
```

Podemos assim utilizar `map` e `nlambda` para definirmos uma função que calcula o `fatorial` de `n` argumentos, memorizando resultados anteriores e devolvendo uma lista: 

```scheme
(define (fast-fact-list . args)
  (let ([memo (make-hash-table)])
    (hash-set! memo 0 1)
    (map (nlambda (fact x)
                  (let ([n (hash-ref memo 0)])
                      (if n
                          n
                          (* x (fact (1- x))))))
         args)))                        
```

Podemos definir `nlambda` em **6** simples linhas:

In [2]:
(defmacro nlambda (n+args . body) 
  (let ([name (car n+args)]       
        [args (cdr n+args)])      
    `(letrec ([,name (lambda ,args
                       ,@body)])  
       ,name)))

### Questão 4

Vamos construir um macro `nlet`, que permite você nomear um bloco `let`, ele possui a seguinte sintaxe:

```scheme
(nlet <nome> ([<var1> <val1>]
              ...
              [<varn> <valn>])
      <corpo>)
```

Caso `<corpo>` chame `<nome>` com `n` argumentos, devemos computar `<corpo>` substituindo cada uma das `n` váriaveis  pelos respectivos `n` argumentos.

Ou seja:

```scheme
(define (fact x)
  (nlet iter ([n x]
              [acc 1])
        (if (zero? n)
            acc
            (iter (1- n) (* n acc)))))
```

É equivalente a uma forma iterativa de computar o fatorial de um número.

##### a) Construa uma função `firsts`, que recebe uma lista de pares `((x 1) (y 2))` e retorna uma lista formada pelo primeiro elemento de cada lista.

In [ ]:
(define (firsts l)
  ...)

##### b) Construa uma função `seconds`, que, equivalente a primeira função, recebe uma lista de pares `((x 1) (y 2))` e retorna uma lista formada pelo segundo  de cada lista.

In [ ]:
(define (map-cadr l)
  ...)
                  

##### c) Por fim, defina o macro `nlet` ☠️.

In [ ]:
(define-macro (nlet nome letargs . body)
  ...)

### Questão 5 ☠️ ☠️
Utilizando `nlet` construa um macro `for` que recebe:

1) uma lista compostas por listas no formato `(<var> <val>)` ou `(<var> <val> <step>)`, no primeiro formato a variável `<var>` mantem seu valor a cada iteração, já no segundo caso `<var>` deve ser substiuido por `(<step> <var>)` a cada iteração;
2) uma lista de condicionais, se todas falsas `#f` a iteração para;
3) um corpo a ser executado a cada iteração.

Por exemplo:
```scheme
(for ([x 0 (1+ x)]
      [y 10])
     [(>= x y)]
  (when (> x 3)
    (display x)
    (newline))
  (when (< y 5)
    (display y)
    (newline)))
```

In [ ]:
(define-macro (for vvs conds . body)
  ...)